In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from konlpy.tag import Twitter

from gensim.models.word2vec import Word2Vec
from os import path, mkdir
from multiprocessing import cpu_count

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# from gensim.test.utils import common_texts,get_tmpfile

import re
import pickle
import time
from math import log10
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\Chankoo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
with open('artTokens.dat','rb') as file: # 미리 doc별로 저장한 tokens의 리스트를 load한다
    tokens = pickle.load(file)

In [66]:
#하이퍼파라미터 세팅
num_features = 20
min_word_count = 3 # 빈도수 3미만 단어 제외
num_workers = cpu_count()
context_size = 10
downsampling = 1e-3
seed=180824

In [67]:
#모델 생성
art2vec =Word2Vec( 
    sg=1, #skip-gram
    workers=cpu_count(),
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample = downsampling,
)

In [68]:
art2vec.build_vocab(tokens)

In [69]:
art2vec.corpus_count

20514

In [39]:
art2vec.epochs

5

In [70]:
art2vec.train(tokens,total_examples=art2vec.corpus_count,epochs=art2vec.epochs) #학습 with tokens

(55750712, 74022550)

In [71]:
##모델을 저장하고 불러와서 다시 training시킬 수 있다
art2vec.save(r'''art2vec_20.w2v''')

In [72]:
##전에 학습한 결과 불러오기
art2vec = Word2Vec.load(r'''art2vec_20.w2v''')

In [73]:
art2vec.wv.most_similar('광주')

[('전당', 0.9533972144126892),
 ('대전', 0.9476805329322815),
 ('시립', 0.946550726890564),
 ('충청', 0.944023072719574),
 ('포항', 0.9380978345870972),
 ('경기도', 0.934951663017273),
 ('호암', 0.9310434460639954),
 ('한일', 0.9299404621124268),
 ('충청남도', 0.9290710091590881),
 ('수교', 0.9281142354011536)]

In [74]:
art2vec.wv.most_similar(['가족','반려동물'])

[('이웃', 0.904069185256958),
 ('사랑하는', 0.8892433047294617),
 ('가정', 0.8864549398422241),
 ('애완동물', 0.8786424398422241),
 ('부모', 0.8691257238388062),
 ('사랑', 0.8685605525970459),
 ('꾸리', 0.8647412061691284),
 ('사람', 0.8629143834114075),
 ('경조사', 0.8566703796386719),
 ('돌보고', 0.8566083908081055)]

In [75]:
art2vec.wv.most_similar('재미')

[('재미있게', 0.9265673160552979),
 ('재미있는', 0.9066023826599121),
 ('재미있고', 0.9008206129074097),
 ('친근하게', 0.8943321704864502),
 ('편안하게', 0.8930883407592773),
 ('다가가며', 0.8849306106567383),
 ('즐겁', 0.8843379020690918),
 ('재미나', 0.8835594654083252),
 ('다가갑', 0.8807575702667236),
 ('색다른', 0.8791844844818115)]

In [76]:
art2vec.wv.most_similar(['최순민','재미'],negative='김섭')

[('친숙하', 0.6541628837585449),
 ('친근하게', 0.6537443399429321),
 ('디저트', 0.6451882123947144),
 ('친근한', 0.6275125741958618),
 ('다가가', 0.6253012418746948),
 ('편하게', 0.617390513420105),
 ('흥미', 0.6143873929977417),
 ('재미있게', 0.609212338924408),
 ('꺼리', 0.5930008888244629),
 ('다가가면', 0.5905745029449463)]

In [77]:
art2vec.wv.most_similar('김혜연')

[('고찬규', 0.8828821182250977),
 ('김경민', 0.8795338869094849),
 ('서유', 0.8783267736434937),
 ('신소영', 0.8773224949836731),
 ('순영', 0.8766965866088867),
 ('이광호', 0.8729751110076904),
 ('하정우', 0.8722391128540039),
 ('홍지연', 0.8702170848846436),
 ('김덕기', 0.8700230121612549),
 ('성지연', 0.8696068525314331)]

In [78]:
art2vec.wv.most_similar('임만혁','김혜연')

[('아무런', 0.7414740324020386),
 ('일어나지', 0.7365439534187317),
 ('명확하지', 0.7259982228279114),
 ('지시하지', 0.7023940086364746),
 ('없거', 0.6939607858657837),
 ('확신할', 0.6919081211090088),
 ('확실하지', 0.6895270943641663),
 ('분명하지', 0.6879034042358398),
 ('불평', 0.6819179058074951),
 ('도무지', 0.6780732870101929)]

In [79]:
gm = (art2vec.wv.get_vector('광주') + art2vec.wv.get_vector('미술관') )/2

In [80]:
h = art2vec.wv.get_vector('희망')

In [81]:
art2vec.wv.cosine_similarities(gm,[h]) # 이런식으로 단어 벡터의 평균을 구해 비교할 수 도 있다

array([0.3551615], dtype=float32)

W2V에 TFIDF로 가중치를 줘보자

In [21]:
with open('TFIDF.dic','rb') as file:
    TFIDF = pickle.load(file)

In [82]:
art2vec.trainables.layer1_size #임베딩 벡터사이즈

20

In [83]:
doc_vecs = []
for i, doc in enumerate(tokens):
    mean_vec = np.zeros((art2vec.trainables.layer1_size,)) # 임베딩 벡터의 사이즈크기로 초기화
    doc_len = len(doc)
    for token in doc:
        try:
            mean_vec += art2vec.wv.get_vector(token) * TFIDF[i][token]
        except Exception as e:
#             print(e)
            doc_len -= 1 # w2v 모델에 없는 voca일 경우 제외하며, 이에따라 doc의 word 수도 줄여준다
            continue
    mean_vec = mean_vec/doc_len # doc의 평균 vector를 구한다
    doc_vecs.append(mean_vec)

In [84]:
doc_vecs[0] #word 벡터의 가중평균인 doc 벡터

array([-0.01110694,  0.11111879, -0.18151247,  0.20661589,  0.17547565,
       -0.23628053, -0.22949162,  0.10842768, -0.31561126,  0.44107216,
        0.18953863, -0.01776567, -0.13022009, -0.15645839, -0.17495093,
       -0.43907461,  0.0608218 , -0.1637886 , -0.18797224, -0.17423725])

query doc을 벡터화시켜서 유사한 doc을 찾아보자

In [85]:
query = """빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 뒤 자신의 귀를 자른 사건 이후 생레미의 요양원에 있을 때 그린 것이다. 반 고흐에게 밤하늘은 무한함을 표현하는 대상이었고, 이보다 먼저 제작된 아를의 <밤의 카페 테라스>나 <론 강 위로 별이 빛나는 밤>에서도 별이 반짝이는 밤의 정경을 다루었다. 반 고흐는 동생 테오에게 쓴 편지에서 “오늘 아침 나는 해가 뜨기 한참 전에 창문을 통해 아무것도 없고 아주 커 보이는 샛별밖에 없는 시골을 보았다.”고 했다. 이 샛별은 그림 가운데 왼쪽에 있는 커다란 흰 별일 것이다. 그가 그린 밤하늘에서는 구름과 대기, 별빛과 달빛이 폭발하고 있다. 황량하고 짙은 파란색 하늘은 세상의 종말을 연상케 하고, 그 위로는 구름이 소용돌이치며 떠있다. 달과 별의 둘레에는 뿌옇게 무리가 져있다.

비연속적이고 동적인 터치로 그려진 하늘은 굽이치는 두꺼운 붓놀림으로 불꽃같은 사이프러스와 연결되고, 그 아래의 마을은 대조적으로 평온하고 고요하다. 마을은 있는 그대로 그린 것이 아니라 부분적으로는 고안되었는데, 교회 첨탑은 반 고흐의 고향 네덜란드를 연상시킨다. 그는 병실 밖으로 내다보이는 밤 풍경을 기억과 상상을 결합시켜 그렸는데, 이는 자연에 대한 반 고흐의 내적이고 주관적인 표현을 구현하고 있다. 수직으로 높이 뻗어 땅과 하늘을 연결하는 사이프러스는 전통적으로 무덤이나 애도와 연관된 나무이지만, 반 고흐는 죽음을 불길하게 보지 않았다. 

그는 “별을 보는 것은 언제나 나를 꿈꾸게 한다.”면서 “왜 하늘의 빛나는 점들에는 프랑스 지도의 검은 점처럼 닿을 수 없을까? 타라스콩이나 루앙에 가려면 기차를 타듯이, 우리는 별에 다다르기 위해 죽는다.”고 했다. 이 시기에 그의 필치는 더욱 두꺼워지고 더욱 열정적으로 변했으며, 꿈틀거리는 듯한 선은 별의 광채를 한층 두드러지게 한다. 이 시기의 특징인 회오리치는 듯 꿈틀거리는 필치는 강렬한 색과 결합되어 감정을 더욱 격렬하게 표현한다. 이 곡선의 화필은 굽이치는 운동감을 표현하면서, 그림 전체를 율동적인 흐름으로 통합한다.
"""

가중치로 쓰기위해 query doc의 TF도 구하자. 문서개수가 하나 뿐인 query doc의 경우 IDF를 구할수 없으므로 빈도수가 곧 가중치가 된다

이는 바람직하지 않으므로 전처리에 신경쓰자

필요한 형태소만 선별하여 가져오고, 2음절 이상 단어만 가져온다. 그리고 불용어 처리까지 해주자

In [86]:
tw = Twitter()
query_tokens = [tag[0] for tag in tw.pos(query) if tag[1] in ['Noun','Adjective','Adverb','Verb','Exclamation'] and len(tag[0]) > 1]
stopwords = ['그','것','이']

In [87]:
query_dic = {}
maxCount = 0

for w in query_tokens:
    if w in stopwords:
        continue
    if w in query_dic.keys():
        query_dic[w] += 1
    else:
        query_dic[w] = 1

    if maxCount < query_dic[w]:
        maxCount = query_dic[w]

print(query_dic)

{'빈센트': 1, '고흐': 6, '대표': 1, '하나로': 1, '꼽히는': 1, '빛나는': 3, '고갱': 1, '다툰': 1, '자신': 1, '자른': 1, '사건': 1, '이후': 1, '레미': 1, '요양원': 1, '그린': 3, '밤하늘': 2, '무한': 1, '표현하는': 1, '대상': 1, '이었': 1, '이보': 1, '먼저': 1, '제작': 1, '카페': 1, '테라스': 1, '위로': 2, '반짝이는': 1, '정경': 1, '다루었': 1, '동생': 1, '테오': 1, '편지': 1, '오늘': 1, '아침': 1, '한참': 1, '창문': 1, '통해': 1, '무것': 1, '없고': 1, '아주': 1, '보이는': 1, '샛별': 2, '없는': 1, '시골': 1, '보았': 1, '그림': 2, '가운데': 1, '왼쪽': 1, '있는': 2, '커다란': 1, '별일': 1, '구름': 2, '대기': 1, '별빛': 1, '달빛': 1, '폭발': 1, '있다': 2, '황량': 1, '짙은': 1, '파란색': 1, '하늘': 4, '세상': 1, '종말': 1, '연상': 2, '하고': 1, '소용돌이': 1, '둘레': 1, '뿌옇': 1, '무리': 1, '연속': 1, '터치': 1, '그려진': 1, '굽이': 2, '치는': 3, '두꺼운': 1, '놀림': 1, '불꽃': 1, '같은': 1, '사이프러스': 2, '연결되': 1, '아래': 1, '마을': 2, '대조': 1, '평온': 1, '고요하다': 1, '그대로': 1, '아니': 1, '부분': 1, '고안': 1, '되었': 1, '교회': 1, '첨탑': 1, '고향': 1, '네덜란드': 1, '시킨': 1, '병실': 1, '내다보': 1, '풍경': 1, '기억': 1, '상상': 1, '결합': 2, '시켜': 1, '그렸': 1, '이는': 1, '자연': 1, '대한': 1, '내적': 1, '주관': 

In [88]:
query_weight = {}
K = 0.5
for k,v in query_dic.items():
    query_weight[k] = K + (1-K) * (v/maxCount)
    print('{0} | {1} + {2} *({3}/{4})  =  {5}'.format(
            k, K, (1-K), v, maxCount, query_weight[k]
        ))

빈센트 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
고흐 | 0.5 + 0.5 *(6/6)  =  1.0
대표 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
하나로 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
꼽히는 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
빛나는 | 0.5 + 0.5 *(3/6)  =  0.75
고갱 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
다툰 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
자신 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
자른 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
사건 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
이후 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
레미 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
요양원 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
그린 | 0.5 + 0.5 *(3/6)  =  0.75
밤하늘 | 0.5 + 0.5 *(2/6)  =  0.6666666666666666
무한 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
표현하는 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
대상 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
이었 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
이보 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
먼저 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334
제작 | 0.5 + 0.5 *(1/6)  =  0.5833333333333334


In [89]:
query_vec = np.zeros((art2vec.trainables.layer1_size,)) # 임베딩 벡터의 사이즈크기로 초기화
query_len = len(query_tokens)

for token in query_tokens:
    try:
        query_vec += art2vec.wv.get_vector(token) * query_weight[token]
    except Exception as e:
            print(e)
            query_len -= 1 # w2v 모델에 없는 voca일 경우 제외하며, 이에따라 query의 word 수도 줄여준다
            continue

query_vec = query_vec/query_len # query의 평균 vector를 구한다
query_vec

"word '사이프러스' not in vocabulary"
"word '사이프러스' not in vocabulary"
"word '타라스' not in vocabulary"


array([-3.32624366e-02,  1.82464302e-01, -1.17285715e-01,  4.73177784e-01,
       -9.68769305e-02, -3.19963781e-01, -2.05094080e-01, -7.52134370e-02,
       -2.34175436e-01,  4.42203144e-01, -1.14770246e-02, -6.71739452e-02,
       -2.71196692e-02, -3.37911989e-02, -2.87056600e-01,  1.47456150e-01,
        5.64000712e-02, -1.49127106e-01, -2.76503564e-01,  3.48939274e-04])

In [90]:
sims = []
for doc_vec in doc_vecs:
    sim = np.dot(query_vec, doc_vec)/(np.linalg.norm(query_vec)*np.linalg.norm(doc_vec))
    sims.append(sim)

In [91]:
sims = np.array(sims)

In [92]:
sims_result = list(zip(np.argsort(sims)[::-1], np.sort(sims)[::-1]))

In [93]:
sims_result[:20]

[(14241, 0.9839799292579733),
 (1588, 0.9824272258479398),
 (18757, 0.9821150302499443),
 (8794, 0.9815979659563902),
 (8827, 0.9813819848896677),
 (1563, 0.9813535605261826),
 (19310, 0.9809557413226639),
 (15260, 0.9808762693994095),
 (13966, 0.9808575296881131),
 (11655, 0.9800779266020068),
 (7538, 0.9797812541432275),
 (16877, 0.9796910693733972),
 (1845, 0.9796320027485496),
 (9171, 0.9796136895389355),
 (10592, 0.9795832501048082),
 (12674, 0.979518726762256),
 (10004, 0.9793781252731174),
 (15446, 0.9789530581867074),
 (9647, 0.978591914888084),
 (6548, 0.9784685628991229)]

In [64]:
d20514 = pd.read_excel('d20514.xlsx',sheet_name='Sheet1')

In [94]:
for idx, sim in sims_result[:10]:
    print(d20514.iloc[idx,:].전시제목,d20514.iloc[idx,:].전시작가,' 유사도: ',sim,'\n',d20514.iloc[idx,:]['링크'])

The epistle of the Others - 소정수展 :: Photography ['소정수(Soh Jungsoo)']  유사도:  0.9839799292579733 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=7499
자연:기억 속의 풍경 - 이종갑展 :: Painting ['이종갑(Lee Jonggab)']  유사도:  0.9824272258479398 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=16583
BLUE - 김태균展 :: Photography ['김태균(Kim Taekyun)']  유사도:  0.9821150302499443 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=12976
소라II 삶-생명의 노래 - 엄길자展 :: Painting ['엄길자(Um Kilja)']  유사도:  0.9815979659563902 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=1224
최병관展 :: Photography ['최병관(Choi Byungkwan)']  유사도:  0.9813819848896677 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=1318
THE LIGHT - 오치균展 :: Painting ['오치균(Oh Chigyun)']  유사도:  0.9813535605261826 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=16923
김준권展 :: Mokpan ['김준권(Kim Joonkwon)']  유사도:  0.9809557413226639 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=13564
항금리 가는 길 - 김인옥展 :: Painting ['김인옥(Kim Inok 金仁玉)']  유사도:  0.980876

artistsDoc to Vec

In [2]:
with open('artists.lst','rb') as file:
    artists_lst = pickle.load(file)

In [30]:
documents = [TaggedDocument(artist[1], [artist[0]]) for artist in artists_lst]

In [31]:
documents[999]

TaggedDocument(words=['인천광역시', '동구', '창', '영동', '7', '번지', '옛', '인천', '양조장', '건물', ',', '배다리', '마을', '골목길', '한', '켠', '에', '스페이스', '빔', '이', '있다', '.', '무겁', '게', '녹이', '슨', '철문', '은', '열려', '있었', '고', '각종', '행사', '를', '알리는', '포스터', '와', '안내', '장', '들이', '붙어', '있었', '을', '뿐', '이었', '다', '.', '언뜻', '무', '뚝뚝', '해', '보였', '다', '.', '아마도', '그', '옆', '에', '서', '있는', '깡통', '로봇', '이', '아니었', '다면', '더', '무', '뚝뚝', '해', '보였', '을지', '모를', '일이', '지만', '그래서', '여느', '장소', '와는', '달리', '그', '안', '을', '더', '들여다보고', '싶은', '호기심', '을', '자극하고', '있었', '다', '.', '철문', '의', '좁', '은', '입구', '들', '지나', '들어선', '스페이스', '빔', '1', '층', '전시장', '은', '전시', '가', '없는', '때', '라서', '말', '그대로', '비어', '있었', '다', '.', '좁', '고', '높다', '란', '계단', '을', '따라', '들어서는', '2', '층', '공간', '은', '카페', '와', '사무실', ',', '연구실', '로', '꾸며', '져', '있고', '민', '운기', '대표', '와', '입', '주', '작가', '들', '이', '무언가', '를', '준비하는', '지', '말', '없이', '분주히', '움직', '이고', '있었', '다', '.', '그리고', '잠시', '시간', '이', '지나자', '스페이스', '빔', '은', '무척', '분주해', '졌', '다', '

In [32]:
max_epochs = 20
vec_size = 100
alpha = 0.025

In [33]:
model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1 #If dm=1, ‘distributed memory’ (PV-DM) is used .Otherwise, distributed bag of words (PV-DBOW) is employed.
               )

In [34]:
model.build_vocab(documents)

In [35]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_artist.model")
print("Model Saved")

iteration 0


C:\Users\Chankoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
Model Saved


In [6]:
model = Doc2Vec.load("d2v_artist.model")

In [36]:
query = """빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 뒤 자신의 귀를 자른 사건 이후 생레미의 요양원에 있을 때 그린 것이다. 반 고흐에게 밤하늘은 무한함을 표현하는 대상이었고, 이보다 먼저 제작된 아를의 <밤의 카페 테라스>나 <론 강 위로 별이 빛나는 밤>에서도 별이 반짝이는 밤의 정경을 다루었다. 반 고흐는 동생 테오에게 쓴 편지에서 “오늘 아침 나는 해가 뜨기 한참 전에 창문을 통해 아무것도 없고 아주 커 보이는 샛별밖에 없는 시골을 보았다.”고 했다. 이 샛별은 그림 가운데 왼쪽에 있는 커다란 흰 별일 것이다. 그가 그린 밤하늘에서는 구름과 대기, 별빛과 달빛이 폭발하고 있다. 황량하고 짙은 파란색 하늘은 세상의 종말을 연상케 하고, 그 위로는 구름이 소용돌이치며 떠있다. 달과 별의 둘레에는 뿌옇게 무리가 져있다.

비연속적이고 동적인 터치로 그려진 하늘은 굽이치는 두꺼운 붓놀림으로 불꽃같은 사이프러스와 연결되고, 그 아래의 마을은 대조적으로 평온하고 고요하다. 마을은 있는 그대로 그린 것이 아니라 부분적으로는 고안되었는데, 교회 첨탑은 반 고흐의 고향 네덜란드를 연상시킨다. 그는 병실 밖으로 내다보이는 밤 풍경을 기억과 상상을 결합시켜 그렸는데, 이는 자연에 대한 반 고흐의 내적이고 주관적인 표현을 구현하고 있다. 수직으로 높이 뻗어 땅과 하늘을 연결하는 사이프러스는 전통적으로 무덤이나 애도와 연관된 나무이지만, 반 고흐는 죽음을 불길하게 보지 않았다. 

그는 “별을 보는 것은 언제나 나를 꿈꾸게 한다.”면서 “왜 하늘의 빛나는 점들에는 프랑스 지도의 검은 점처럼 닿을 수 없을까? 타라스콩이나 루앙에 가려면 기차를 타듯이, 우리는 별에 다다르기 위해 죽는다.”고 했다. 이 시기에 그의 필치는 더욱 두꺼워지고 더욱 열정적으로 변했으며, 꿈틀거리는 듯한 선은 별의 광채를 한층 두드러지게 한다. 이 시기의 특징인 회오리치는 듯 꿈틀거리는 필치는 강렬한 색과 결합되어 감정을 더욱 격렬하게 표현한다. 이 곡선의 화필은 굽이치는 운동감을 표현하면서, 그림 전체를 율동적인 흐름으로 통합한다.
"""

In [37]:
tw = Twitter()
query_tokens = tw.morphs(query)

In [39]:
new_vector = model.infer_vector(query_tokens,steps=100) #
sims = model.docvecs.most_similar([new_vector])
sims

[('김은숙', 0.6585316061973572),
 ('배병우', 0.6315444707870483),
 ('김선두', 0.6277011632919312),
 ('도성욱', 0.6159570813179016),
 ('박성진', 0.6123545169830322),
 ('김호득', 0.6102941036224365),
 ('임만혁', 0.6022536754608154),
 ('변관식', 0.5968416333198547),
 ('노수현', 0.596458375453949),
 ('박노수', 0.5945721864700317)]

In [43]:
[' '.join(artist[1]) for artist in artists_lst if artist[0]=='김은숙']

["국립 현대 미술관 서울 관 ( 관장 정형민 ) 은 4 월 18 일 부터 27 일 까지 2014 창작 스튜디오 입 주 작가 를 소개하는 《 INTRO 》 전 을 개최 한 다 . 국립 현대 미술관 창작 스튜디오 는 고양 과 창동 ( 2 곳 ) 에 위치하고 있으 며 , 국내 시각예술 작가 들 의 창작 여건 을 활성화 시키 고자 1 년 간 의 장기 입 주 프로그램 을 통해 젊고 유망 한 작가 들 을 선발 하여 지원하고 있다 . 《 INTRO 》 전 은 창작 스튜디오 에 새로 둥지 를 튼 작가 를 소개하기 위해 해마다 개최 되는 정례 전이 다 . 이번 전시 는 현대 미술 의 실험 현장 인 창작 스튜디오 에서 역동 적 인 움직임 을 보여 줄 24 명 작가 들 의 대표 작 을 선 보인 다 . 회화 , 설치 , 영상 등 다양한 분야 의 작품 을 만날 수 있으 며 , 진취 적 인 예술가 들 의 실험 적 작업 을 위한 공간 으로 마련된 서울 관 교육 동의 워크숍 갤러리 에서 전시되 어 그 의미 를 더한 다 . 이번 에 선정 된 24 명의 창작 스튜디오 입 주 작가 는 공개 모집 및 2 회의 심사 과정 을 거쳐 선발 되었 다 . 이 들 은 한국 현대 미술 을 선도 할 인재 임 을 인정받 았 으며 , 향후 국립 현대 미술관 의 지원 과 함께 역량 을 키워 나갈 예정 이다 . 전시 기간 중 에는 관람객 들 이 예술가 들 의 작업 세계 를 보다 폭넓 게 이해하고 이 들 과 직접 소통 할 수 있도 록 하는 프로그램 , ‘ 입 주 작가 프리젠테이션 ’ 을 4 월 22 , 23 일 양일간 진행한 다 . 작가 뿐 아니 라 미술 계 전문가 ( 미술 평론가 정현 , 월간 아티 클 편집장 홍경 한 등 ) 도 참여해 함께 이야기 를 나눌 예정 이다 . ⓒ 예술가 가 주체 가 되 고 , 집단 이 되어 전시 공간 을 습격 하는 본 전시 는 많은 것 이 모 일수록 그 어떠한 의미 구현 이 모호 해 질 수 있는 현재 블록버스터 급 의 전시 를 비롯한 멀티 복합 문화 스타일 의 전시 구현 현상 에 대한 

도서 서평을 넣어보자

In [ ]:
query = """“사람들이 나보고 맘충이래.”

한국에서 여자로 살아가는 일
그 공포, 피로, 당황, 놀람, 혼란, 좌절의
연속에 대한 인생 현장 보고서 

조남주 장편소설 『82년생 김지영』이 민음사 ‘오늘의 젊은 작가’ 시리즈로 출간되었다. 조남주 작가는 2011년, 지적 장애가 있는 한 소년의 재능이 발견되면서 벌어지는 사건을 통해 삶의 부조리를 현실적이면서도 따뜻하게 그려낸 작품 『귀를 귀울이면』으로 ‘문학동네소설상’을 받으며 데뷔했다. 시사 교양 프로그램에서 10년 동안 일한 방송 작가답게 서민들의 일상에서 발생하는 비극을 사실적이고 공감대 높은 스토리로 표현하는 데 특출 난 재능을 보이는 작가는 신작 『82년생 김지영』에서 30대를 살고 있는 한국 여성들의 보편적인 일상을 완벽하게 재현한다.
주인공 ‘김지영 씨’의 기억을 바탕으로 한 고백을 한 축으로, 고백을 뒷받침하는 각종 통계 자료와 기사들을 또 다른 축으로 삼는 이 소설은 1982년생 김지영 씨로 대변되는 ‘그녀’들의 인생 마디마디에 존재하는 성차별적 요소를 핍진하게 묘사한다. 이를 통해 작가는 제도적 성차별이 줄어든 시대의 보이지 않는 차별들이 어떻게 여성들의 삶을 제약하고 억압하는지 보여 준다. 
여권이 신장된 시대, 그러나 여전히 ‘여성’이라는 조건이 굴레로 존재하는 사회에서 살아가는 한 여자의 인생을 다룬 『82년생 김지영』은 조용한 고백과 뜨거운 고발로 완성된 새로운 페미니즘 소설이자 수많은 사람들의 경험과 자료로 이루어진 ‘목소리 소설’이다. 맘충이, 여혐, 메갈리아 등 연일 새롭게 등장하는 페미니즘 화두를 관심 있게 지켜보는 독자라면 누구나 흥미롭게 읽을 수 있고 저마다 의미를 발견할 수 있을 것이다. 

■“사람들이 나보고 맘충이래.” 
엄마를 뜻하는 ‘맘(Mom)’과 벌레를 뜻하는 ‘충(蟲)’의 합성어인 ‘맘충’은 제 아이만 싸고도는 일부 몰상식한 엄마를 가리키는 용어다. 그러나 ‘맘충’이란 호칭은 육아하는 엄마 대부분에게 무차별적으로 사용되며 많은 여성들에게 공포심을 주고 상처를 안겼다. 뿐만 아니라 이 표현은 육아가 마치 여성의 일인 것처럼 인식되게 함으로써 성차별적 시선을 고착화하는 데도 일조해 논란의 대상이 되었다. 
『82년생 김지영』은 2014년 말 촉발된 ‘맘충이’ 사건을 목격한 작가가 여성, 특히 육아하는 여성에 대한 사회의 폭력적인 시선에 충격 받아 쓰기 시작한 소설이다. 소설을 쓸 당시 작가는 유치원 다니는 자녀를 둔 전업주부였다. 온라인상에서 사실 관계도 확인되지 않은 상황만 놓고 엄마들을 비하하는 태도에 문제의식을 느낀 작가는 지금 한국을 살아가는 여성들의 삶이 과거에서 얼마나 더 진보했는지, 혹은 그렇지 않은지 질문할 수 있는 이야기를 만들기로 했다. 

■30대 여성들의 인생 보고서
슬하에 딸을 두고 있는 서른네 살 김지영 씨가 어느 날 갑자기 이상 증세를 보인다. 시댁 식구들이 모여 있는 자리에서 친정 엄마로 빙의해 속말을 뱉어 내는 통에 시댁 식구들을 아연실색하게 만드는가 하면 남편의 결혼 전 애인으로 빙의해 그를 식겁하게 만들기도 한다. 이를 이상하게 여긴 남편이 김지영 씨의 정신 상담을 주선하고, 지영 씨는 정기적으로 의사를 찾아가 자신의 삶을 이야기한다.
소설은 김지영 씨의 이야기를 들은 담당 의사가 그녀의 인생을 재구성해 기록한 리포트 형식이다. 리포트에 기록된 김지영 씨의 기억은 ‘여성’이라는 젠더적 기준으로 선별된 에피소드로 구성된다. 발화의 기회가 주어졌을 때 그녀가 선택한 이야기들이 바로 일생에 거쳐 ‘여자이기 때문에 받아 왔던 부당한 일들’이기 때문이다. 
이러한 개인의 고백은 1999년 남녀차별을 금지하는 법안이 제정되고 이후 여성부가 출범함으로써 성평등을 위한 제도적 장치가 마련된 이후, 즉 제도적 차별이 사라진 시대에 보이지 않는 방식으로 존재하는 내면화된 성차별적 요소가 작동하는 방식을 보여 준다. 지나온 삶을 거슬러 올라가며 미처 못다 한 말을 찾는 이 과정은 지영 씨를 알 수 없는 증상으로부터 회복시켜 줄 수 있을까?

■김지영으로 대변되는 젊은 여성들에 대한 섬세한 심리 묘사 
상담은 자기 고백 형식으로 이루어진다. 이 소설의 백미도 김지영 씨의 자기 고백을 중심으로 드러나는 세밀한 심리 묘사다. ‘그때 그 상황’에서는 차마 말하지 못했던 것들을 차분히 쏟아 내는 그녀의 말들은 ‘김지영’을 이 시대 여성의 대변자로 삼기에 충분할 정도로 자세하고 보편적이다. 더욱이 김지영의 이름은 이 시대 젊은 여성들의 삶을 보편적으로 그리기 위한 작가의 전략적 선택이기도 하다. 실제로 1982년에 태어난 여아 중 가장 많이 등록된 이름이 ‘지영’이기 때문이다. 김지영이라는 개인의 고백을 30대 여성, 나아가 이 시대 여성들의 고백으로 볼 수 있는 이유다. 

선배는 평소와 똑같이 다정하고 차분히 물었다. 껌이 무슨 잠을 자겠어요, 라고 대답하고 싶었지만 김지영 씨는 입을 다물어 버렸다. (94쪽)

영업 중인 빈 택시 잡아 돈 내고 타면서 고마워하기라도 하라는 건가. 배려라고 생각하며 아무렇지도 않게 무례를 저지르는 사람. 어디서부터 어디까지 항의를 해야 할지도 가늠이 되지 않았고, 괜한 말싸움을 하기도 싫어 김지영 씨는 그냥 눈을 감아 버렸다. (100~101쪽)

주량을 넘어섰다고, 귀갓길이 위험하다고, 이제 그만 마시겠다고 해도 여기 이렇게 남자가 많은데 뭐가 걱정이냐고 반문했다. 니들이 제일 걱정이거든. 김지영 씨는 대답을 속으로 삼키며 눈치껏 빈 컵과 냉면 그릇에 술을 쏟아 버렸다. (116쪽)

조금도 서운하지 않았다. 견딜 수 없는 것은 오히려 그 순간들이었다. 김지영 씨는 충분히 건강하다고, 약 같은 것은 필요 없다고, 가족 계획은 처음 보는 친척들이 아니라 남편과 둘이 하겠다고 말하고 싶었다. 하지만 아니에요, 괜찮아요, 라는 말밖에 할 수 없었다. (133~134쪽)


■기사, 통계, SNS 등 풍부하고 탄탄한 취재
보고서 형식으로 쓰인 『82년생 김지영』의 에피소드들은 무척이나 사실적이다. 어린 시절, 학창 시절, 회사 생활, 결혼 생활에 이르기까지 여성이라면 누구에게나 익숙한 이 경험들은 온라인 커뮤니티와 SNS에서 많은 공감대를 형성한 사례들을 채집한 결과이기 때문이다. 이러한 구체적인 사례들과 함께 등장하는 각종 팩트들은 지난 20여 년 동안의 ‘성차별 역사’를 한눈에 보여 준다. 『확률 가족』 『기록되지 않은 노동』 『고용 동향 브리프』 등의 도서와 「여자라고 전교 회장 못 하나요」 등의 신문 기사를 비롯해 「인구 동태 건수 및 동태율」 「출산 순위별 출생 성비」 같은 통계청 자료, OECD에서 발표한 성별 인금 격차 (Gender wage gap) 자료 및 외신 기사, 「호주제 페지: 호주제, 벽을 넘어 평등 세상으로」 등 행정부 정책 보고서, 「경력단절 여성 지원정책의 현황과 과제」 같은 보건복지포럼 등의 자료가 쉴 새 없이 등장한다. 개인적 기억과 고백의 형식을 취하고 있는 이야기는 이러한 사실적 자료들을 통해 한국에서 살아가는 여성의 보편적인 삶으로 도약하는 근거가 된다. 


추천글 하나도 낯설지가 않은데 새삼 눈물이 고이다니 이상한 일이다. 눈 돌릴 수 없는 통계와 보도 사이, 그리고 눈에 보이지도 않을 만큼 미세한 차별과 폭력 속에 성장한 나와 내 또래 수많은 지영이들의 삶에 대한 담담하고 서글픈 보고서. 어차피 해피엔딩은 오지 않을 것이다. 다만 운 좋게, 혹은 우연히 살아남은 '여아'들이었던 우리는 이렇게 말하고 기록을 남길 수밖에. -최지은(웹진 《아이즈》 기자)"""

In [ ]:
tw = Twitter()
query_tokens = tw.morphs(query)

In [65]:
new_vector = model.infer_vector(query_tokens,steps=100) #
sims = model.docvecs.most_similar([new_vector])
sims

[('유목연', 0.5955752730369568),
 ('김보경', 0.5832899808883667),
 ('홍지철', 0.580819308757782),
 ('김승택', 0.5807056427001953),
 ('위영일', 0.5754429697990417),
 ('홍인숙', 0.5731268525123596),
 ('나광호', 0.5728030800819397),
 ('이정민', 0.569983959197998),
 ('이동근', 0.5658045411109924),
 ('송윤주', 0.5648694038391113)]

In [66]:
[' '.join(artist[1]) for artist in artists_lst if artist[0]=='유목연']

["Sex + Pleasure 남녀 의 육체 적 결합 행위 의 뜻 을 내포 하고 있는 성 ( 性 , sex ) 은 기본 적 으로 ‘ 쾌락 ’ 을 수 반한 다 . 성 행위 의 목적 이 애정 의 표현 이든 , 성욕 의 충족 이든 , 생식 이든지 간 에 성적 쾌락 을 추구 하 는 것 은 식욕 과 수면 욕 에 더불어 인간 의 가장 ‘ 원초 적 ’ 본능 이다 . Sex + Guilty 인간 의 기본 적 욕구 에 의한 - 심지어 즐거움 을 선사 하는 - ‘ 정당 한 ’ 행위 인 성은 사회 적 규범 안 에서 단순한 성적 쾌락 의 추구 로부터 자유롭 지 못 하게 된 다 . 성규 범 이 생기 게 된 이유 들 을 되 짚어 보면 유교 적 전통 , 기독교 적 신념 , 가족 을 이루는 기본 단위 로서 의 결혼 제도 , 그리고 그 에 따른 도덕 적 가치판단 등 이 있다 . 또한 성규 범의 또 다른 이면 에는 사회 의 권력관계 속 에서 지배 층 의 위계 질서 의 확립 을 위해 성적 쾌락 을 통제 하고 , 여기 에 동성애 와 여성 에게는 이중 적 잣 대가 더해 져 조건 없는 노동 력 확보 와 가부장 적 구조 를 합리화 시키 기도 한 다 . 성적 규제 에 의해 ‘ 성 = 불건전 한 행위 ’ 로 치부 되 기 쉬 우 며 이 것 은 성 을 드러내 놓 는 것 이 꺼려 지는 이유 이기도 하다 . 그 결과 사회 적 규범 안 에서 성적 쾌락 의 추구 는 종종 ‘ 죄 의식 ’ 을 동반 하는 데 아이러니 하게 도 이러한 규범 이 만들어 내 는 ‘ 금기 ’ 는 다시 쾌락 의 바탕 이 된 다 . Sex + Guilty Pleasure 성규 범 이 존재함 에도 불구하고 한국 의 성문 화 는 급속도 로 개방 적 으로 변화해 가 고 있다 . 여성 의 혼전순결 이 당연시 되 던 것 이 지금 은 오히려 숫총각 , 숫처녀 들 이 어딘가 문제 있는 ‘ 천연기념물 ’ 취급 을 당한 다 . 인터넷 세상 은 어떠한 가 ? 그야말로 성의 ‘ 판타지 랜드 ’ 이다 . 이렇게 변해 가 는 성문 화 에 비해 성규 범 은 여전히 제자리